In [2]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
from torch.utils.data import DataLoader, Dataset
import torch
from data_import import *

In [3]:
oregon_data_dict = oregon_import()

file import:   0%|          | 0/3 [00:00<?, ?it/s]

file import: 100%|██████████| 3/3 [00:02<00:00,  1.48it/s]


In [4]:
wa_df = oregon_data_dict['train_timeseries'].iloc[4:,:].reset_index(inplace=False, drop=True)
wa_df = wa_df[wa_df['fips']==41067]

In [5]:
X_1 = wa_df.iloc[:7, (wa_df.columns != 'score') & (wa_df.columns != 'date')]
y_1 = wa_df.iloc[6, wa_df.columns == 'score']
date_1 = wa_df.iloc[:7, wa_df.columns == 'date']

In [6]:
X_2 = wa_df.iloc[7:14, (wa_df.columns != 'score') & (wa_df.columns != 'date')]
y_2 = wa_df.iloc[13, wa_df.columns == 'score']
date_2 = wa_df.iloc[7:14, wa_df.columns == 'date']

In [7]:
class DroughtDataset(Dataset):
    """
    Readying Drought dataset for model.
    """
    def __init__(self, df):
        self.X = torch.tensor(df.iloc[:, (df.columns != 'score') & (df.columns != 'date')].values)
        self.y = torch.tensor(df.iloc[:, df.columns == 'score'].dropna().values)
        self.date = np.array(df.iloc[:, df.columns == 'date'].values)

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        output_X = self.X[7*index:7+7*index]
        output_y = self.y[6 + 7*index]
        output_date = self.date[7*index:7+7*index]

        return output_X, output_y, output_date

In [8]:
wa_dataset = DroughtDataset(wa_df)

In [9]:
import torch.nn as nn
torch.manual_seed(1)

In [10]:
class LSTM_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTM_Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h_0=None, c_0=None):
        if h_0 is None or c_0 is None:
            h_0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
            c_0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        
        out, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])
        return out, h_n, c_n